In [1]:
import numpy as np
import sympy as sp
from sympy import *
from sympy import Matrix
import matplotlib.pyplot as pyplot
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
from numpy import linalg
import cmath
#%matplotlib inline
from mpl_toolkits import mplot3d
import time
#import numba
#from numba import jit

In [2]:
##### CURVATURE #############################################

def U_calc(v1,v2): # trick of the FHS method
    U = np.vdot(v1,v2)
    return U/abs(U)


##### CALCULATION OF BRAKETS #####
def braket(v1,observable,v2):
    return np.vdot(v1,np.dot(observable,v2))


##### TORUS PERIODIC HOFSTATDER HAMILTONIAN #####

def H_Hofstatder_tore(kx,ky,p,q):
    
    phi = p/q
    J = 1
    a = 1
    
    H = 1j*np.zeros((q,q))
    for i in range(q):
        H[i,i] = -J*2*np.cos(kx*a+i*2*np.pi*phi) 
        if (i>0 and i<q): 
            H[i,i-1] = -J
            H[i-1,i] = J      
            
    H[0,-1] = -J*np.exp(1j*q*a*ky)
    H[-1,0] = -J*np.exp(-1j*q*a*ky)    
    
    return H

##### ANALYTICAL DERIVATIVE OF H-H HAMILTONIAN #####

def dH_d_Hof(kx,ky,p,q): # for Hofstatder hamiltonian
    
    dH_dkx = 1j*np.zeros((q,q))
    dH_dky = 1j*np.zeros((q,q))
    a = 1
    J = 1
    
    for i in range(q):
        dH_dkx[i,i] = 2*J*a*np.sin(a*kx+i*2*np.pi*p/q)
        #if (i>0 and i<q):    # Attention: there is a difference betwenn ref Hafezi-GOldman vs Dalibard.
         #   dH_dky[i,i-1] = 1j*J*a*np.exp(-1j*ky*a)   
          #  dH_dky[i-1,i] = -1j*J*a*np.exp(1j*ky*a)
    
    dH_dky[0,-1] = -q*J*1j*a*np.exp(q*1j*a*ky)
    dH_dky[-1,0] = q*J*1j*a*np.exp(-q*1j*a*ky)
    
    return dH_dkx,dH_dky

##### FHS CURVATURE FOR H-H MODEL #####


def F_Hof(H,n,kx,ky,dkx,dky,p,q): # algorithm based on the FHS method for a single plaquette
    ''' Inputs: Hamiltonian of the system under considerations (function of kx,ky),
                Coordinates of a point in the Brillouin Zone (BZ),
                Dimensions of one plaquette of the BZ.
        Outputs: Berry curvature at point (kx,ky) in the BZ'''
    
    eigV_0 = linalg.eigh(H_Hofstatder_tore(kx,ky,p,q))[1][:,n]
    eigV_1 = linalg.eigh(H_Hofstatder_tore(kx+dkx,ky,p,q))[1][:,n]
    eigV_2 = linalg.eigh(H_Hofstatder_tore(kx+dkx,ky+dky,p,q))[1][:,n]
    eigV_3 = linalg.eigh(H_Hofstatder_tore(kx,ky+dky,p,q))[1][:,n]
    
    U_01 = U_calc(eigV_0,eigV_1)
    U_12 = U_calc(eigV_1,eigV_2)
    U_23 = U_calc(eigV_2,eigV_3)
    U_30 = U_calc(eigV_3,eigV_0)
    
    return (np.log(U_01*U_12*U_23*U_30)/dkx/dky).imag

##### NIU CURVATURE FOR H-H MODEL #####


def F_Niu_Hof(H_Hofstatder_tore,n,kx,ky,p,q): # calculates the Berry curv in kx,ky by Niu method
    
    F_Niu_n = 0 
    eigValues = linalg.eigh(H_Hofstatder_tore(kx,ky,p,q))[0]
    eigVectors = linalg.eigh(H_Hofstatder_tore(kx,ky,p,q))[1]
    En = linalg.eigh(H_Hofstatder_tore(kx,ky,p,q))[0][n]
    eigVectors_n = linalg.eigh(H_Hofstatder_tore(kx,ky,p,q))[1][:,n]
    
    deriv_H = dH_d_Hof(kx,ky,p,q)
    dH_dkx = deriv_H[0]
    dH_dky = deriv_H[1]
    
    for n_prime in range(len(eigVectors)):
        if (n_prime!=n):
            eigVectors_n_prime = eigVectors[:,n_prime]
            En_prime = eigValues[n_prime]
            
            temporary1 = braket(eigVectors_n,dH_dkx,eigVectors_n_prime)
            temporary2 = braket(eigVectors_n_prime,dH_dky,eigVectors_n)
            F_Niu_n +=  (temporary1*temporary2-np.conj(temporary1*temporary2))/(En-En_prime)**2/1j
            
    return F_Niu_n

##### CHERN NUMBER CALCULATION #####

def chern(n,F_Hof,kx_list,ky_list,dkx,dky,p,q):
    chern_n = 0
    for kx in kx_list:
        for ky in ky_list:
            chern_n += F_Hof(H_Hofstatder_tore,n,kx,ky,dkx,dky,p,q)*dkx*dky
            
    return chern_n

##################################################################################################



##### HOFSTATDER CYLINDRICAL ############################

##### EDGES STATES BY TRANSFER MATRIX #####

def En_edge(E,ky,p,q,Ly,Phi):
    
    phi = p/q
    r = 1 # r = J_1/J_2
    
    M = np.eye(2)
    for m in range(1,q+1):
        M = Matrix([[-E-2*r*np.cos(ky-2*np.pi*(Phi/Ly+phi*m)+np.pi),-1],[1,0]])*M 
    eigValues_M = solve(M[1,0],E)
    
    return eigValues_M

##### CYLINDRICAL PERIODIC HOFSTATDER HAMILTONIAN #####

def H_Hofstatder_cylind(Lx,Ly,ky,p,q,Phi): 
    tx = 1
    ty = 1
    phi = p/q
    
    H_ky = np.zeros((Lx,Lx))
    
    for m in range(Lx):    # m is the real space coordinate and goes from 0 to Lx-1. The function range doen't take the last input in the created vector
                             
            
       # Next one is from the Bernevig's book and Hatsugai's paper
        H_ky[m,m] = -2*ty*np.cos(ky-2*np.pi*phi*(m-1/2)-2*np.pi*Phi/Ly) # Attention to this extra term 1/2!!!!!!!!!
        
        if (m<Lx-1):
            H_ky[m,m+1] = -tx
            H_ky[m+1,m] = -tx
                
    return H_ky

###############################################################################################

# HOFSTADTER OPEN BOUNDARIES

def H_Hofstatder_square(tx,ty,p,q,positions_x,positions_y,Lx,Ly):    # construct a matrix with Lx bloc matrices and each of them is Ly x Ly matrices
    
    phi = p/q
    H = 1j*np.zeros((Lx*Ly,Lx*Ly))
    
    for m in positions_x:
        for n in positions_y:
            
            if (m<Lx-1):
                H[n+(m+1)*Lx,n+m*Lx] = -tx
                H[n+m*Lx,n+(m+1)*Lx] = -tx
                
            if (n<Ly-1):
                H[(n+1)+m*Lx,n+m*Lx] = -ty*np.exp(1j*2*np.pi*phi*m)
                H[n+m*Lx,(n+1)+m*Lx] = -ty*np.exp(-1j*2*np.pi*phi*m)
        
    return H

# DYNAMICS

def vector2matrix(vector): # takes a vector of n^2 lines and return a matrix n x n
    l = int(np.sqrt(len(vector)))
    matrix = 1j*np.zeros((l,l))
    k = 0
    for i in range(l):
        for j in range(l):
            matrix[i,j] = vector[k]
            k += 1
    return matrix
# test: vector2matrix(np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]))

def matrix2vector(matrix):
    N1 = len(matrix)
    N2 = len(matrix[0])
    vector = 1j*np.zeros(N1*N2)
    k = 0
    for i in range(N1):
        for j in range(N2):
            vector[k] = matrix[i,j]
            k += 1
    return vector

def delta_init(x0,y0,Lx,Ly):
    init_state = np.zeros((Ly,Lx))
    init_state[y0,x0] = 1
    
    return init_state


def Gauss_init(Lx,Ly,x0,y0,ky,sigma): # sigma=1 is the smallest choice

    init_state = 1j*np.zeros((Ly,Lx))
    
    for x in range(Lx):
        for y in range(Ly):
            init_state[y,x] = np.exp(-((x-x0)**2 + (y-y0)**2)/2/sigma**2-1j*ky*y)
    
    return init_state


def overlap(init_state,edge_states):
    out = 1j*edge_states.copy()
    for i in range(len(out)):
        for j in range(len(out[0])):
            out[j,i] = abs(np.conjugate(init_state[j,i])*edge_states[j,i])**2
    return out